In [40]:
import toml
import pandas as pd

import itertools
import numpy as np
import re
import logging

from goes2go.data import goes_latest, goes_nearesttime, goes_timerange

log = logging.getLogger(__name__)

In [33]:
product_table = pd.DataFrame(toml.load("./product_table.toml")).T

_products = []
for i, j in product_table.iterrows():
    if isinstance(j.domains, list):
        a = list(itertools.product([i], j.domains))
        _products.append([''.join(i) for i in a])
    else:
        _products.append([i])
_products = set(itertools.chain(*_products))

# you can be unspecific and request any mesoscale domain (M), 
# or by number (M1, M2)
_domains = {"F", "C", "M", "M1", "M2"} 

In [22]:
_products

{'ABI-L1b-RadC',
 'ABI-L1b-RadF',
 'ABI-L1b-RadM',
 'ABI-L2-ACHAC',
 'ABI-L2-ACHAF',
 'ABI-L2-ACHAM',
 'ABI-L2-ACHTC',
 'ABI-L2-ACHTF',
 'ABI-L2-ACHTM',
 'ABI-L2-ACMC',
 'ABI-L2-ACMF',
 'ABI-L2-ACMM',
 'ABI-L2-ACTPC',
 'ABI-L2-ACTPF',
 'ABI-L2-ACTPM',
 'ABI-L2-ADPC',
 'ABI-L2-ADPF',
 'ABI-L2-ADPM',
 'ABI-L2-AICEF',
 'ABI-L2-AITAF',
 'ABI-L2-AODC',
 'ABI-L2-AODF',
 'ABI-L2-BRFC',
 'ABI-L2-BRFF',
 'ABI-L2-BRFM',
 'ABI-L2-CMIPC',
 'ABI-L2-CMIPF',
 'ABI-L2-CMIPM',
 'ABI-L2-CODC',
 'ABI-L2-CODF',
 'ABI-L2-CPSC',
 'ABI-L2-CPSF',
 'ABI-L2-CPSM',
 'ABI-L2-CTPC',
 'ABI-L2-CTPF',
 'ABI-L2-DMWC',
 'ABI-L2-DMWF',
 'ABI-L2-DMWM',
 'ABI-L2-DMWVC',
 'ABI-L2-DMWVF',
 'ABI-L2-DMWVM',
 'ABI-L2-DSIC',
 'ABI-L2-DSIF',
 'ABI-L2-DSIM',
 'ABI-L2-DSRC',
 'ABI-L2-DSRF',
 'ABI-L2-DSRM',
 'ABI-L2-FDCC',
 'ABI-L2-FDCF',
 'ABI-L2-FDCM',
 'ABI-L2-LSAC',
 'ABI-L2-LSAF',
 'ABI-L2-LSAM',
 'ABI-L2-LSTC',
 'ABI-L2-LSTF',
 'ABI-L2-LSTM',
 'ABI-L2-LVMPC',
 'ABI-L2-LVMPF',
 'ABI-L2-LVMPM',
 'ABI-L2-LVTPC',
 'ABI-L2-LVTPF',

In [37]:
class GOES:
    def __init__(self, satellite, product, domain=None):
        self.satellite = satellite
        self.product = product
        self.domain = domain

        self._check_satellite
        self._check_product

    @property
    def _check_satellite(self):
        if isinstance(self.satellite, int):
            self.satellite = f"noaa-goes{self.satellite}"
        elif isinstance(self.satellite, str):
            self.satellite = re.sub("[^0-9]", "", self.satellite)
            self.satellite = f"noaa-goes{self.satellite}"

    @property
    def _check_product(self):
        if self.product == "GLM":
            # Alias for geostationary lighting mapper
            self.product = "GLM-L2-LCFA"
        elif self.product == "ABI":
            # Alias for multi-channel cloud moisture image
            if self.domain is None:
                self.product = "ABI-L2-MCMIP" + "C"
            else:
                self.product = "ABI-L2-MCMIP" + self.domain

        if self.product.startswith("ABI"):
            if self.domain is None:
                if self.product[0] in _domains:
                    self.domain = self.product[0]

            elif self.domain is not None:
                if self.domain not in _domains:
                    raise ValueError(
                        f"domain for ABI products must be None or one of {_domains}"
                    )
        else:
            if self.domain is not None:
                log.warning("domain argument is ignored for non-ABI products")

    def __repr__(self):
        msg = [
            f"--- This GOES Object ---",
            f"{self.satellite=}",
            f"{self.product=}",
            f"{self.domain=}",
        ]
        return "\n".join(msg)

    def latest(self, **kwargs):
        return goes_latest(
            satellite=self.satellite,
            product=self.product,
            domain=self.domain,
            **kwargs
        )

    def nearest(self, attime, within, **kwargs):
        return goes_latest(
            attime,
            within,
            satellite=self.satellite,
            product=self.product,
            domain=self.domain,
            **kwargs
        )

    def goes_timerange(self, start, end, recent, **kwargs):
        return goes_timerange(
            start, end, recent, 
            satellite=self.satellite,
            product=self.product,
            domain=self.domain,
            **kwargs
        )


In [38]:
GOES('goes16', 'ABI', "F").latest()

 _______________________________
 | Satellite: noaa-goes16      |
 |   Product: ABI-L2-MCMIPF    |
 |    Domain: F                |
📦 Finished downloading [1] files to [C:\Users\blayl_depgywe\data\noaa-goes16\ABI-L2-MCMIPF].


,file,product_mode,satellite,start,end,creation
0,noaa-goes16/ABI-L2-MCMIPF/2022/127/05/OR_ABI-L...,ABI-L2-MCMIPF-M6,G16,2022-05-07 05:00:20.600,2022-05-07 05:09:52.500,2022-05-07 05:10:01.700
